In [2]:
import importlib.metadata
import pathlib
import pandas as pd
import numpy as np
import secrets
import hashlib

import anywidget
import traitlets
import arkouda as ak
import arachne as ar
import numpy as np
import pandas as pd
import networkx as nx
from compcon.create_graph import get_neuron_local
from fafbseg import flywire

    _         _                   _       
   / \   _ __| | _____  _   _  __| | __ _ 
  / _ \ | '__| |/ / _ \| | | |/ _` |/ _` |
 / ___ \| |  |   < (_) | |_| | (_| | (_| |
/_/   \_\_|  |_|\_\___/ \__,_|\__,_|\__,_|
                                          

Client Version: v2024.04.19


In [3]:
import arkouda as ak
import pandas as pd
from vimosketches.src.anywidget_test import Widget
from skelescope.skelescope import Skelescope
from compcon.create_graph import get_neuron, get_graph, draw_graph, get_neuron_local, overall, draw_connection, draw3d_graph
from fafbseg import flywire

In [4]:
def get_segment_idx_nds(final_segment_paths, paths_with_slabs):
    offset = 0
    highlighted_path = []
    for path in final_segment_paths:
        for segment_idx, segment in enumerate(path):
            for pws in paths_with_slabs:
                for p in pws:
                    if segment == p:
                        highlighted_path.append(offset + segment_idx)
                        
        offset += len(path)
    return highlighted_path

# This function takes two lists, `list1` (the "host") and `list2` (the "sub").
# It checks if `list2` appears within `list1` and returns `list1` split around `list2`:
# - Example 1: list1 = [1, 2, 3, 4], list2 = [1, 2] => Output: [2, 3, 4]
# - Example 2: list1 = [1, 2, 3, 4], list2 = [2, 3] => Output: ([1, 2], [3, 4])
# - Example 3: list1 = [1, 2, 3, 4], list2 = [3, 4] => Output: [1, 2, 3]

def remove_subsequence_if_exists(list1, list2):
    # Find the starting index of list2 in list1
    n, m = len(list1), len(list2)
    
    # Iterate over list1 to find a matching subsequence
    for i in range(n - m + 1):
        if list1[i:i + m] == list2:  # Check if list2 matches list1 starting at index i
            if i==0:
                # print(list1[i + m-1:])
                return list1[i + m-1:]
            
            if (i+m) == len(list1):
                return list1[:i+1]
            
            return list1[:i+1], list1[i + m-1:]  # Remove the subsequence
    
    return list1

def modify_neuron_segments(neuron_segments, paths_with_slabs):
    temp_storage = []
    for pws in paths_with_slabs:
        for ns in neuron_segments:
            result = remove_subsequence_if_exists(ns, pws)
            if type(result) == tuple:
                #FIX THE PROBLEM
                neuron_segments[neuron_segments.index(ns)] = result[1]
                temp_storage.append(result[0])
            
            else:
                neuron_segments[neuron_segments.index(ns)] = result
            
    return neuron_segments+temp_storage+paths_with_slabs 

def get_final_segment_paths(neuron_segments, paths_with_slabs):
    
    final_segment_paths=[]
    for index, path in enumerate(paths_with_slabs):
        temp_path = []
        temp_path = modify_neuron_segments(neuron_segments[index], path)
        final_segment_paths.append(temp_path)

    return final_segment_paths

def normalize_column(column):
    return (column) / (1000)

def normalize_neuron_list(not_ds_neurons):
    neuron_nodes = []
    neuron_edges = []
    neuron_segments = []
    for n in not_ds_neurons:
        n.nodes['x'] = normalize_column(n.nodes['x'])
        n.nodes['y'] = normalize_column(n.nodes['y'])
        n.nodes['z'] = normalize_column(n.nodes['z'])

        # n.connectors['x'] = normalize_column(n.connectors['x'])
        # n.connectors['y'] = normalize_column(n.connectors['y'])
        # n.connectors['z'] = normalize_column(n.connectors['z'])

        n.nodes.radius = 0.1
        neuron_nodes.append(n.nodes)
        neuron_edges.append(n.edges)
        neuron_segments.append(n.segments)
    
    return neuron_nodes, neuron_edges, neuron_segments

def get_slabs(skeleton_edges, n_ids):
    paths_with_slabs = []
    unique_n_ids = set(n_ids)
    path_ids = []
    not_ds_neurons = []

    for index, id in enumerate(unique_n_ids):
        n = get_neuron_local(id, 3000)
        nx_display = nx.Graph()
        nx_display.add_edges_from(n.edges)
        
        shortest_path=[]
        temp_path=[]
        for edge_index, edge in enumerate(skeleton_edges):
            if n_ids[edge_index]==id:
                start_node_id = skeleton_edges[edge_index][0]
                end_node_id = skeleton_edges[edge_index][1]
                try:
                    shortest_path = nx.shortest_path(nx_display, source=start_node_id, target=end_node_id, weight='weight')
                    
                except nx.NetworkXNoPath:
                    print(f"No path found between and")

                temp_path.append(shortest_path)
                path_ids.extend([id])
        not_ds_neurons.append(n)
        paths_with_slabs.append(temp_path)
    return paths_with_slabs, path_ids, not_ds_neurons

def back_transformation_edges(d, src_mapped, dst_mapped):
    skeleton_edges = []
    neuron_ids = []
    for i in range(len(src_mapped)):
        df_temp = d[(d['src'] == src_mapped[i]) & (d['dst'] == dst_mapped[i])][0]
        if df_temp["connection_type"] == 'n':
            new_element = [df_temp["s_bef"], df_temp["s_af"]]
            # wenn edges in verschiedenen neurons gleiche ids haben geht das nicht!!(check if neuron_ids übereinstimmen!)
            if new_element not in skeleton_edges:
                skeleton_edges.append(new_element)
                neuron_ids.append(int(df_temp["n_id"]))

            new_element = [df_temp["d_bef"], df_temp["d_af"]]
            if new_element not in skeleton_edges:
                skeleton_edges.append(new_element)
                neuron_ids.append(int(df_temp["n_id"]))

    return skeleton_edges, neuron_ids

def mapping_edges(subgraph, mapping):
    edges = subgraph.edges()
    src = edges[0].to_ndarray()
    dst = edges[1].to_ndarray()

    src = src.astype(str)
    dst = dst.astype(str)

    src_mapped = np.vectorize(mapping.get)(src)
    dst_mapped = np.vectorize(mapping.get)(dst)

    return src_mapped, dst_mapped

def test_graph_creation(edges):
    first_elements = edges[:, 0]
    second_elements = edges[:, 1]
    neighbor_matrix = (first_elements[:, np.newaxis] == second_elements[np.newaxis, :])

    # Step 3: Extract neighbor pairs
    neighbor_indices = np.argwhere(neighbor_matrix)
    neighbor_indices = neighbor_indices[neighbor_indices[:, 0] != neighbor_indices[:, 1]]
    neighbor_pairs = edges[neighbor_indices]

    # Step 4: Combine node indices to create unique identifiers for edges
    src_combined = neighbor_pairs[:, 0, 0] * 10000 + neighbor_pairs[:, 0, 1]
    dst_combined = neighbor_pairs[:, 1, 0] * 10000 + neighbor_pairs[:, 1, 1]

    # Convert to lists for Arkouda
    src = src_combined.astype(np.int64).tolist()
    dst = dst_combined.astype(np.int64).tolist()
    return src, dst

def drawing_transformation(branches, synapses):
    nodes=[]
    branch_type= []

    for b in branches:
        nodes.append(b[0] * 10000 + b[1])

    src, dst = test_graph_creation(branches)
    for s in src:
        branch_type.append("n")

    for s in synapses:
        src.append(s[0][0] * 10000 + s[0][1])
        dst.append(s[1][0] * 10000 + s[1][1])
        branch_type.append("s")
    
    return src, dst, branch_type

def overfinal(d, motif, mapping):
    ak.connect()
    branches, synapses = [], []
    for m in motif:
        if m["properties"][0] == "neuron connection":
            branches.append(m["label"])
            
        if m["properties"][0] == "synaptic connection":
            synapses.append([m["properties"][1], m["properties"][2]])
        # print(type((m["properties"][0])))
    
    branches = np.array(branches)
    synapses = np.array(synapses)

    src, dst, branch_type = drawing_transformation(branches, synapses)
    
    dicts = {
        "src": src,
        "dst": dst,
        "connection_type": branch_type
        }
    subgraph = ar.PropGraph()
    df = ak.DataFrame(dicts)
    subgraph.load_edge_attributes(df, source_column="src", destination_column="dst", 
                                relationship_columns=["connection_type"])
    src_mapped, dst_mapped = mapping_edges(subgraph, mapping)
    
    skeleton_edges, n_ids = back_transformation_edges(d, src_mapped, dst_mapped)
   
    paths_with_slabs, path_ids, not_ds_neurons = get_slabs(skeleton_edges, n_ids)
    
    neuron_nodes, neuron_edges, neuron_segments = normalize_neuron_list(not_ds_neurons)
    
    org_n = []
    add_edges = []
    
    for nds in not_ds_neurons:
        n = get_neuron_local(nds.id)
        org_n.append(n)
        # Convert sublists to tuples to use set operations
        n_segments_set = set(map(tuple, n.edges))
        np_segments_set = set(map(tuple, nds.edges))
        
        # Find the difference
        difference_set = n_segments_set - np_segments_set
    
        # Convert the result back to a list of lists
        difference = list(map(list, difference_set))
        add_edges.append(difference)
    
    o_neuron_nodes, o_neuron_edges, o_neuron_segments = normalize_neuron_list(org_n)
    
    final_segment_paths = get_final_segment_paths(neuron_segments, paths_with_slabs)

    for j in range(len(add_edges)):
        final_segment_paths[j] = final_segment_paths[j] + add_edges[j]
        # paths_with_slabs[j] = paths_with_slabs[j] + add_edges[j]
        
        
    
    final_segment_paths = [[j for j in i if len(j) >= 2] for i in final_segment_paths]
    
    highlighted_paths = get_segment_idx_nds(final_segment_paths, paths_with_slabs)
    highlighted_paths_twigs = get_segment_idx_nds(final_segment_paths, add_edges)
    
    ak.disconnect()

    return o_neuron_nodes, final_segment_paths, highlighted_paths, highlighted_paths_twigs

In [5]:
df = pd.read_csv("/home/michaelshewarega/Desktop/test/demo_output.csv")
ak.connect()
transformed_dataset = ak.DataFrame(df.to_dict(orient='list'))

connected to arkouda server tcp://*:5555


In [6]:
motif_json = [{'label': [0, 1],
  'properties': ['neuron connection', '#00880A'],
  'index': 0,
  'indices': [0, 1],
  'tree': None},
 {'label': [1, 2],
  'properties': ['neuron connection', '#00880A'],
  'index': 1,
  'indices': [1, 2],
  'tree': None},
 {'label': [3, 4],
  'properties': ['neuron connection', '#003090'],
  'index': 2,
  'indices': [3, 4],
  'tree': None},
 {'label': [4, 5],
  'properties': ['neuron connection', '#003090'],
  'index': 3,
  'indices': [4, 5],
  'tree': None},
 {'label': [6, 7],
  'properties': ['synaptic connection', [1, 2], [4, 5]],
  'index': 4,
  'indices': [6, 7],
  'tree': None},
 {'label': [8, 9],
  'properties': ['synaptic connection', [0, 1], [3, 4]],
  'index': 5,
  'indices': [8, 9],
  'tree': None}]

# current_mapping = sketching_board.current_mapping
current_mapping = {'1': 41793983, '10002': 39833628, '30004': 26552418, '40005': 24181930}


In [7]:
transformed_dataset[(transformed_dataset["src"] == 39833628) & (transformed_dataset["dst"] == 24181930)][0]["d_x"]

751428.0

In [14]:
neuron_nodes, final_segment_paths, highlighted_paths, highlighted_paths_twigs =  overfinal(transformed_dataset, motif_json, current_mapping)

ak.connect()
x = transformed_dataset[(transformed_dataset["src"] == current_mapping["1"]) & (transformed_dataset["dst"] == current_mapping["30004"])][0]["d_x"]
y = transformed_dataset[(transformed_dataset["src"] == current_mapping["1"]) & (transformed_dataset["dst"] == current_mapping["30004"])][0]["d_y"]
z = transformed_dataset[(transformed_dataset["src"] == current_mapping["1"]) & (transformed_dataset["dst"] == current_mapping["30004"])][0]["d_z"]
x2 = transformed_dataset[(transformed_dataset["src"] == current_mapping["10002"]) & (transformed_dataset["dst"] == current_mapping["40005"])][0]["d_x"]
y2 = transformed_dataset[(transformed_dataset["src"] == current_mapping["10002"]) & (transformed_dataset["dst"] == current_mapping["40005"])][0]["d_y"]
z2 = transformed_dataset[(transformed_dataset["src"] == current_mapping["10002"]) & (transformed_dataset["dst"] == current_mapping["40005"])][0]["d_z"]
data = {
    'x': [x, x2],    # Wrap the list in an extra list to create a single row
    'y': [y, y2],
    'z': [z, z2],
    'type': ["pre", "pre"],
    'node_id': [-1, -1]
}

# Create the DataFrame
con = pd.DataFrame(data)
con['x'] = normalize_column(con['x'])
con['y'] = normalize_column(con['y'])
con['z'] = normalize_column(con['z'])

connected to arkouda server tcp://*:5555
disconnected from arkouda server tcp://*:5555
connected to arkouda server tcp://*:5555


In [15]:
con

,x,y,z,type,node_id
0,754.418,204.222,145.08,pre,-1
1,751.428,200.194,145.54,pre,-1


In [17]:
viewer = Skelescope()
viewer.add_neuron(neuron_nodes, con, final_segment_paths, ["green", "blue", "red"], highlighted_paths, highlighted_paths_twigs)
viewer

Skelescope(axis_local_primary_points=[0, 0, 0, 0, 1, 0], segments={0: {'parent_segment': -1, 'children_segment…

In [1]:
sketching_board.current_mapping

NameError: name 'sketching_board' is not defined

In [4]:
sketching_board = Widget(arkouda_df=transformed_dataset)
sketching_board

Widget()

In [56]:
np= get_neuron_local(720575940631103007, prune_factor=5000)
n= get_neuron_local(720575940631103007)

In [ ]:
import numpy as np

# Sample data
x_bef, y_bef, z_bef = np.array([0, 0, 0])
x_af, y_af, z_af = np.array([10, 10, 10])

# Define interpolation ratios
ratios = np.linspace(0, 1, num=10)  # 10 evenly spaced points between 0 and 1

# Generate sampling points for each coordinate
s_x = [(r * x_bef + (1 - r) * x_af).tolist() for r in ratios]
s_y = [(r * y_bef + (1 - r) * y_af).tolist() for r in ratios]
s_z = [(r * z_bef + (1 - r) * z_af).tolist() for r in ratios]

print("s_x:", s_x)
print("s_y:", s_y)
print("s_z:", s_z)
